### Import Modules

In [7]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time

sys.path.append("../")
from fitter import data_loader as dl
from fitter import bootstrapper as bs
from fitter import fitter as fit
from fitter import special_functions as sf

matplotlib.rcParams['figure.figsize'] = [10, 10]

In [104]:
reload(bs)
reload(dl)
reload(fit)
reload(sf)

<module 'fitter.special_functions' from '../fitter\special_functions.pyc'>

### Specify fit parameters

In [105]:
p_dict = {
    'order' : {
        'fit' : 'nlo', # 'nlo', 'nnlo', or 'nnnlo'
        'latt_spacing' : 2, # no order 1 term -- starts at 2
        'vol' : 8 # max 10
    }, 
    'bs_N' : 1,  # if 0, use full list
    'fit_type' : 'simultaneous', #'ma', 'ma-taylor, 'xpt', or 'xpt-taylor'; also 'simultaneous', 'ma-old'
    'F2' : 'FKFpi', #'FKfpi', 'FpiFpi', 'FKFK'
    
    'use_prior' : True,
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m310', u'a12m350',
           u'a12m400', u'a15m220', u'a15m310', u'a15m350', 'a15m400'],
    
    'make_plots' : True,
    'show_error_ellipses' : False,
    'show_bs_histograms' : False,
    
    'save_prior' : False,
    'save_results' : False,
}

### Do fit

In [106]:
t0 = time.time()

# Load data
data_loader = dl.data_loader()
fit_data = data_loader.get_fit_data()

# Get prior
prior = None
if p_dict['use_prior']:
    prior = data_loader.get_prior(p_dict['fit_type'])

# Make bootstrapper
bootstrapper = bs.bootstrapper(fit_data, prior=prior, order=p_dict['order'], F2=p_dict['F2'],
                               fit_type=p_dict['fit_type'], bs_N=p_dict['bs_N'], abbrs=p_dict['abbrs'])

if p_dict['make_plots']:
    data_loader.save_plots(
        bootstrapper.make_plots(
            show_error_ellipses=p_dict['show_error_ellipses'], 
            show_bootstrap_histograms=p_dict['show_bs_histograms']),
        output_filename=bootstrapper.fit_type+'_'+bootstrapper.order['fit']
    )
else:
    print bootstrapper
    
if p_dict['save_prior']:
    print 'Saving prior...'
    data_loader.save_prior(bootstrapper.create_prior_from_fit(), p_dict['fit_type'])
    
if p_dict['save_results']:
    data_loader.save_fit_info(bootstrapper.get_fit_info())
    
    
t1 = time.time()

print "\nTotal time (s): ", t1 - t0, "\n"

Making fits...
100% complete Time (s):  2.95800018311
Compiling results...
Done.

Total time (s):  8.60299992561 



In [107]:
print bootstrapper.fits[0]

Least Square Fit:
  chi2/dof [dof] = 68 [28]    Q = 0    logGBF = -809.34

Parameters:
          mpi 0     0.098068 (66)       [ 0.097900 (67) ]  **
              1      0.14102 (12)       [  0.14072 (12) ]  **
              2      0.15807 (19)       [  0.15785 (19) ]  *
              3      0.18119 (14)       [  0.18116 (14) ]  
              4      0.08142 (25)       [  0.08126 (25) ]  
              5      0.13446 (17)       [  0.13428 (17) ]  *
              6      0.13612 (31)       [  0.13557 (32) ]  *
              7      0.18889 (17)       [  0.18870 (17) ]  *
              8      0.21411 (20)       [  0.21397 (20) ]  
              9      0.24357 (16)       [  0.24347 (16) ]  
             10      0.16538 (19)       [  0.16533 (19) ]  
             11      0.23613 (30)       [  0.23601 (30) ]  
             12      0.26483 (29)       [  0.26473 (29) ]  
             13      0.30292 (30)       [  0.30281 (30) ]  
           mk 0     0.228526 (98)       [ 0.228702 (99) ]  *
    

### Or make all fits

In [ ]:
p_dict = {
    'order' : {
        'latt_spacing' : 2, # no order 1 term -- starts at 2
        'vol' : 8 # max 10
    }, 
    'bs_N' : 1,  # if 0, use full list
    'F2' : 'FKFpi', #'FKfpi', 'FpiFpi', 'FKFK'

    'use_prior' : True,
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m310', u'a12m350',
           u'a12m400', u'a15m220', u'a15m310', u'a15m350', 'a15m400'],

    'make_plots' : True,
    'show_error_ellipses' : False,
    'show_bs_histograms' : False,

    'save_prior' : False,
    'save_results' : True,
}

In [90]:
for fit_type in ['xpt', 'xpt-taylor', 'ma', 'ma-taylor']:
    for order_fit in ['nlo', 'nnlo', 'nnnlo']:
        
        print fit_Type, order_fit
        
        # Specify fit parameters
        p_dict['order']['fit'] = order_fit
        p_dict['fit_type'] = fit_type
        
        # Do fit
        t0 = time.time()

        # Load data
        data_loader = dl.data_loader()
        fit_data = data_loader.get_fit_data()

        # Get prior
        prior = None
        if p_dict['use_prior']:
            prior = data_loader.get_prior(p_dict['fit_type'])

        # Make bootstrapper
        bootstrapper = bs.bootstrapper(fit_data, prior=prior, order=p_dict['order'], F2=p_dict['F2'],
                                       fit_type=p_dict['fit_type'], bs_N=p_dict['bs_N'], abbrs=p_dict['abbrs'])

        if p_dict['make_plots']:
            data_loader.save_plots(
                bootstrapper.make_plots(
                    show_error_ellipses=p_dict['show_error_ellipses'], 
                    show_bootstrap_histograms=p_dict['show_bs_histograms']),
                output_filename=bootstrapper.fit_type+'_'+bootstrapper.order['fit']
            )
        else:
            print bootstrapper

        if p_dict['save_prior']:
            print 'Saving prior...'
            data_loader.save_prior(bootstrapper.create_prior_from_fit(), p_dict['fit_type'])

        if p_dict['save_results']:
            data_loader.save_fit_info(bootstrapper.get_fit_info())


        t1 = time.time()

        print "\nTotal time (s): ", t1 - t0, "\n"

Making fits...
100% complete Time (s):  0.364999771118
Compiling results...
Done.
Saving...
Done.

Total time (s):  5.40900015831 

Making fits...
100% complete Time (s):  0.382999897003
Compiling results...
Done.
Saving...
Done.

Total time (s):  5.37699985504 

Making fits...
100% complete Time (s):  0.406000137329
Compiling results...
Done.
Saving...
Done.

Total time (s):  5.63899993896 

Making fits...
100% complete Time (s):  0.348000049591
Compiling results...
Done.
Saving...
Done.

Total time (s):  5.35100007057 

Making fits...
100% complete Time (s):  0.333999872208
Compiling results...
Done.
Saving...
Done.

Total time (s):  5.48699998856 

Making fits...
100% complete Time (s):  0.373000144958
Compiling results...
Done.
Saving...
Done.

Total time (s):  5.55900001526 

Making fits...
100% complete Time (s):  0.893000125885
Compiling results...
Done.
Saving...
Done.

Total time (s):  6.65600013733 

Making fits...
100% complete Time (s):  1.01699995995
Compiling results...
D

In [92]:
#bootstrapper.plot_parameter_histogram('L_5')

In [86]:
#print bootstrapper

In [87]:
#print bootstrapper.fits[0]

In [220]:
#print bootstrapper.fits[0]

In [214]:
#bootstrapper.fits[0].show_plots()

In [34]:
#print bootstrapper.fits[0]